# Imports needed for HMM training and evaluation

In [5]:
# from helper import accuracy, entity_count
%pip install datasets
import nltk
from sklearn.metrics import precision_recall_fscore_support
from datasets import load_dataset, load_metric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00


# Retrieve dataset

In [6]:
from datasets import load_dataset, load_metric
dataset = load_dataset("surrey-nlp/PLOD-CW")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1072 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/126 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/153 [00:00<?, ? examples/s]

In [7]:
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

In [109]:
# trainSentences = train_dataset["tokens"]
trainSentences = dataset["train"][:]["tokens"]
trainLabels = dataset["train"][:]["ner_tags"]

print(len(trainSentences))

print(trainSentences)
print(trainLabels)

1072
[['For', 'this', 'purpose', 'the', 'Gothenburg', 'Young', 'Persons', 'Empowerment', 'Scale', '(', 'GYPES', ')', 'was', 'developed', '.'], ['The', 'following', 'physiological', 'traits', 'were', 'measured', ':', 'stomatal', 'conductance', '(', 'gs', ',', 'mol', 'H2O', 'm-2', 's-1', ')', ',', 'transpiration', 'rate', '(', 'E', ',', 'mmol', 'H2O', 'm-2', 's-1', ')', ',', 'net', 'photosynthetic', 'rate', '(', 'PN', ',', 'μmol', 'm-2', 's-1', ')', 'and', 'intercellular', 'CO2', 'concentration', 'CO2', '(', 'Ci', ',', 'μmol', 'm-2', 's-1', ')', '.'], ['Minor', 'H', 'antigen', 'alloimmune', 'responses', 'readily', 'occur', 'in', 'the', 'setting', 'of', 'human', 'leukocyte', 'antigen', '(', 'HLA)–matched', 'allogeneic', 'solid', 'organ', 'and', 'stem', 'cell', 'transplantation', '(', 'SCT', ')', '[', '3,4', ']', '.'], ['EPI', '=', 'Echo', 'planar', 'imaging', '.'], ['Furthermore', ',', 'eNOS', '-', 'derived', 'NO', 'S', '-', 'nitrosylated', 'β', '-', 'actin', 'on', 'Cys374', 'and', 'impai

In [110]:
# We need the unique set of tokens
# We need the unique set of labels (pre-post conversion?)

symbolSet = set()
for tokenSet in trainSentences:
  for token in tokenSet:
    for char in token:
      symbolSet.add(char)

# Get the unique set of labels
entitySet = set()
for tagSet in trainLabels:
  for tag in tagSet:
    entitySet.add(tag)

In [111]:
entitySet = list(entitySet) # Convert back to lists
print(entitySet)

symbolSet = list(symbolSet)
print(symbolSet)
print(len(symbolSet))

['B-O', 'B-LF', 'B-AC', 'I-LF']
['ν', '7', 'C', '3', 'F', '{', 'j', '2', 'X', 'Φ', 'c', 'Q', '’', '(', 'l', '>', '）', 'β', '$', 'θ', 'í', '-', '‒', '≥', 'D', '•', 'p', 'n', ';', '…', '[', '§', 'O', '´', '=', '9', 'f', 'H', ']', 'J', 'ß', 'I', '6', 'E', 'V', 'Ä', 'W', '/', 'g', 'U', '‘', 'N', '×', '8', 'λ', 'Ω', 'q', '（', '5', 'R', '≤', '_', 'ï', '"', '†', '&', '–', '+', 'ε', ',', '%', 'o', '4', 'σ', 'δ', '→', '‡', 'M', '@', '”', '±', '®', '*', '}', 'B', 'Y', '0', '?', 'y', '°', 'a', 's', 'G', 'Δ', 'ö', '<', 'z', '∞', 'P', 'ã', 'k', 'κ', 'e', 'w', ':', '♀', 'µ', 'ü', 'γ', 'ú', 'x', 'L', '“', '#', 'è', 'K', '♂', 'Å', '¯', ')', 'h', 'S', 'μ', 't', 'T', "'", 'r', '∑', 'Z', '′', 'd', '1', 'α', 'v', '·', 'A', 'i', 'm', '—', 'ó', '−', '.', 'b', 'u', 'é']
145


In [30]:
# Flatten the training data for easy use later
flat_inputs_train = [item for sublist in train_dataset["tokens"] for item in sublist]
flat_labels_train = [item for sublist in train_dataset["ner_tags"] for item in sublist]

# Creating a function to give us a combined set of training tokens and their corresponding label for it

Our dataset needs to be a list of lists

In [112]:
# trainCombined = [(token, label) for token, label in zip(flat_inputs_train, flat_labels_train)]

# trainCombined = [(token, label) for tokenSet, labelSet in zip(trainSentences, trainLabels) for token, label in zip(tokenSet, labelSet)]

trainCombinedOld = [[(token, label) for token, label in zip(tokenSet, labelSet)] for tokenSet, labelSet in zip(trainSentences, trainLabels)]

trainCombined = []
for tokens, labels in zip(trainSentences, trainLabels):
    sequence = [(token, label) for token, label in zip(tokens, labels)]
    trainCombined.append(sequence)

print(trainCombined[0])
print(trainCombinedOld[0])

[('For', 'B-O'), ('this', 'B-O'), ('purpose', 'B-O'), ('the', 'B-O'), ('Gothenburg', 'B-LF'), ('Young', 'I-LF'), ('Persons', 'I-LF'), ('Empowerment', 'I-LF'), ('Scale', 'I-LF'), ('(', 'B-O'), ('GYPES', 'B-AC'), (')', 'B-O'), ('was', 'B-O'), ('developed', 'B-O'), ('.', 'B-O')]
[('For', 'B-O'), ('this', 'B-O'), ('purpose', 'B-O'), ('the', 'B-O'), ('Gothenburg', 'B-LF'), ('Young', 'I-LF'), ('Persons', 'I-LF'), ('Empowerment', 'I-LF'), ('Scale', 'I-LF'), ('(', 'B-O'), ('GYPES', 'B-AC'), (')', 'B-O'), ('was', 'B-O'), ('developed', 'B-O'), ('.', 'B-O')]


# Setting up and training HMM

In [113]:
trainerSetup = nltk.tag.hmm.HiddenMarkovModelTrainer(states = entitySet, symbols = symbolSet)

model = trainerSetup.train_supervised(trainCombined)

# Evaluating

In [121]:
# valSentences = val_dataset["tokens"]
# valLabels = val_dataset["ner_tags"]
import numpy as np

flatValSentences = [item for sublist in val_dataset["tokens"] for item in sublist]
flatValLabels = [item for sublist in val_dataset["ner_tags"] for item in sublist]

# print(val_dataset["tokens"][0])

# print(np.array(flatValSentences).shape)

# print(flatValSentences)

# valSentences = []
# for tokenSet in val_dataset["tokens"]:
#   for item in tokenSet:
#     valSentences += item

test_sentences = dataset["test"][:]["tokens"]

# Predict with HMM, you need to tag each token individually rather than pass in the entire list otherwise it will predict everything as one item!
# preds = []
# for token in flatValSentences:
#   preds.append(model.tag(token))

predicted = []
for sentence in test_sentences:
    test_result = model.tag(sentence)
    out_tags = []
    for word, tag in test_result:
        out_tags.append(tag)
    predicted.append(out_tags)
predicted[0]

['B-O',
 'B-O',
 'B-O',
 'B-O',
 'B-O',
 'B-O',
 'B-O',
 'B-O',
 'B-O',
 'B-O',
 'B-O',
 'B-O',
 'B-O',
 'B-O',
 'B-O']

In [129]:
#print(list(predicted))

# print(flatValSentences)
# print(preds)

predTags = [tag for tagList in predicted for tag in tagList]

print(predTags)

print(len(predTags))

['B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O'

In [120]:
# # Getting just the tags out
# predictedTags = []

# for resultSet in preds:
#   #print(resultSet)
#   for tup in resultSet:
#     print(tup)
#     predictedTags.append(tup[1])

# print(len(predictedTags))

Streaming output truncated to the last 5000 lines.
('n', 'B-O')
('s', 'B-O')
('u', 'B-O')
('s', 'B-O')
('i', 'B-O')
('n', 'B-O')
('g', 'B-O')
('a', 'B-O')
('m', 'B-AC')
('o', 'B-O')
('d', 'B-O')
('i', 'B-O')
('f', 'B-O')
('i', 'B-O')
('e', 'B-O')
('d', 'B-O')
('c', 'B-O')
('e', 'B-O')
('t', 'B-O')
('y', 'B-O')
('l', 'I-LF')
('t', 'B-O')
('r', 'B-O')
('i', 'B-O')
('m', 'B-AC')
('e', 'B-O')
('t', 'B-O')
('h', 'B-O')
('y', 'B-O')
('l', 'I-LF')
('a', 'B-O')
('m', 'B-O')
('m', 'B-AC')
('o', 'B-O')
('n', 'B-O')
('i', 'B-O')
('u', 'B-O')
('m', 'B-O')
('b', 'B-O')
('r', 'B-O')
('o', 'B-O')
('m', 'B-O')
('i', 'B-O')
('d', 'B-O')
('e', 'B-O')
('(', 'B-O')
('C', 'B-O')
('T', 'B-O')
('A', 'B-O')
('B', 'B-O')
(')', 'B-O')
('m', 'B-AC')
('e', 'B-O')
('t', 'B-O')
('h', 'B-O')
('o', 'B-O')
('d', 'B-O')
('.', 'B-O')
('T', 'B-O')
('h', 'B-O')
('e', 'B-O')
('m', 'B-AC')
('e', 'B-O')
('d', 'B-O')
('i', 'B-O')
('a', 'B-O')
('n', 'B-O')
('a', 'B-O')
('g', 'B-O')
('e', 'B-O')
('a', 'B-O')
('t', 'B-O')
('f', 

In [80]:
print(flatValLabels)

['B-O', 'B-LF', 'I-LF', 'I-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-AC', 'B-O', 'B-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-LF', '

In [135]:
from sklearn.metrics import classification_report
# classification_report(flatValLabels, predictedTags)

# print(len(flatValLabels))
# print(len(predTags))

# print(set(predTags))

print(classification_report(flatValLabels, predTags))

              precision    recall  f1-score   support

        B-AC       0.08      0.01      0.01       263
        B-LF       0.00      0.00      0.00       149
         B-O       0.85      0.98      0.91      4261
        I-LF       0.03      0.00      0.01       327

    accuracy                           0.84      5000
   macro avg       0.24      0.25      0.23      5000
weighted avg       0.73      0.84      0.78      5000

